Давайте решим следующую задачу.<br>
Необходимо написать робота, который будет скачивать новости с сайта Лента.Ру и фильтровать их в зависимости от интересов пользователя. От пользователя требуется отмечать интересующие его новости, по которым система будет выделять области его интересов.<br>


Начнем с загрузки новостей. Для этого нам потребуется метод requests.get(url). Библиотека requests предоставляет серьезные возможности для загрузки информации из Интернет. Метод get получает URL стараницы и возвращает ее содержимое. В нашем случае результат будет получаться в формате html. <br>
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Теперь попробуем загрузить страницу новостей.

In [2]:
# Для пробы получаем первую страницу сайта.
requests.get("https://lenta.ru/")

<Response [200]>

Метод <i>requests.get()</i> возвращает объект Response, который содержит большое количество различной информации о загруженной (или незагруженной) странице. В краткой форме отображается только результат выполения запроса. В нашем случае это 200, нет ошибки.<br> 
Посмотрим что результат содержит еще.

In [3]:
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")

In [4]:
dir(resp)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [5]:
%%time
# %%time - Магия Jupyter - замеряет время выполнения ячейки. Должно быть первой строчкой в ячейке.
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie is_mobile=0 for .lenta.ru/>, <Cookie lid=vAsAAKfzYWTeZYPBAXyZXwB= for .lenta.ru/>, <Cookie lids=482065DE7CE9F95E for .lenta.ru/>]>
time to download: 0:00:00.997520
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2018/08/24/clon/
CPU times: user 71.8 ms, sys: 0 ns, total: 71.8 ms
Wall time: 1.21 s


Но самое для нас интересное хранится в поле <i>text</i>, которое содержит собственно текст html-страницы.

In [8]:
#Берем первые 1000 символов новости.
resp.text[:1000]

'<!DOCTYPE html><html lang="ru"><head><title>В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru</title><meta charset="utf-8" /><meta content="#292929" name="theme-color" /><link href="https://m.lenta.ru/news/2018/08/24/clon/" media="only screen and (max-width: 640px)" rel="alternate" /><link href="https://lenta.ru/rss/google-newsstand/main/" rel="alternate" type="application/rss+xml" /><link href="https://lenta.ru/news/2018/08/24/clon/" rel="canonical" /><link href="/manifest.json" rel="manifest" /><link rel="shortcut icon" type="image/x-icon" href="https://icdn.lenta.ru/favicon.ico" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-256x256.png" size="256x256" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-192x192.png" size="192x192" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-152x152.png" size="

Количество служебной информации в странице явно превышает объем текста новости. У нас есть два пути: либо использовать библиотеку BeautyfulSoup для получения текста статьи, либо получить текст с использованием регулярных выражений.

Опробуем первый путь. Документация на библиотеку BeautyfulSoup находится <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">здесь</a>.

В ячейке ниже мы создаем объект BeautifulSoup, передаем в него текст html-страницы и сообщаем, что разбирать его надо при помощи библиотеки `html5lib`. Далее просим отдать текст страницы без html-тегов.

In [9]:
BeautifulSoup(resp.text, "html5lib").get_text()[:1000]

'В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru{"@context":"http://schema.org","@type":"NewsArticle","headline":"В Сибири нашли подходящих для клонирования древних животных","description":"Палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Подобные находки имеют большое количество сохранившейся ДНК, которая подходит для клонирования.","name":"В Сибири нашли подходящих для клонирования древних животных","alternativeHeadline":"","url":"https://lenta.ru/news/2018/08/24/clon/","mainEntityOfPage":{"@type":"WebPage","@id":"https://lenta.ru/news/2018/08/24/clon/"},"thumbnailUrl":"https://icdn.lenta.ru/images/2018/08/24/13/20180824130540171/detail_fb4ef24e26e8448a2a23747f000a4489.jpg","dateCreated":"2018-08-24T15:22:00+03:00","datePublished":"2018-08-24T15:22:00+03:00","dateModified"

Да, убрать html-теги получилось. Но их содержимое осталось, в том числе и скрипты.<br>
Опробуем другой путь. Весь текст обычно оформляется тегом параграфа - &lt;p&gt;. Выберем весь текст из этих тегов. Заодно выберем и заголовок статьи, оформленный при помощи \<h1\>.

In [10]:
# Получили объект BeautifulSoup и скормили ему текст страницы.
bs = BeautifulSoup(resp.text, "html5lib") 
# Вот таким образом можно попросить отдать первый тег, отмеченный как h1. Вместо h1 можно написать любой другой тег.
title = bs.h1.text
# Получаем все параграфы (тег p), берем их текст без тегов и склеиваем в один текст.
text = " ".join([p.text for p in bs.find_all("p")])
print(title, "\n-----\n", text)

В Сибири нашли подходящих для клонирования древних животных 
-----
 Фото: AP Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного. У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было. У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех останков древних лошадей. В 2015 году в Я

Теперь напишем функцию, которая выгружает все новости за сутки. <br>
Обратим внимание, что для сайта Lenta.ru можно написать адрес в формате lenta.ru/ГГГГ/ММ/ДД/ (год, месяц, день) и получить все новости за этот день. Попробуем получить все адреса с такой страницы.

In [9]:
# Идем на страницу, получаем ее текст, отдаем в BeautifulSoup, ищем все теги ссылок - а.
BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib").find_all("a")[:20]

[<a class="menu__nav-link _is-extra" href="/">Главное</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/russia/">Россия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/world/">Мир</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/ussr/">Бывший СССР</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/economics/">Экономика</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/forces/">Силовые структуры</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/science/">Наука и техника</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/culture/">Культура</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/sport/">Спорт</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/media/">Интернет и СМИ</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/style/">Ценности </a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/travel/">Путешествия</a>,
 <a class="menu__nav-link _is-extra" href="/rubrics/life/">Из жизни</a>,
 <a class="menu__nav-lin

Кажется, это опять немного не то, что нам нужно. Мы получили все ссылки, находящиеся на боковом меню, ссылки на события сегодняшнего дня и другие ненужные нам вещи. <br>
Смотрим в содержимое html-страницы и обращаем внимание, что все интересные нам ссылки оформлены как заголовки третьего уровня - &lt;h3&gt;. Извлечем все такие фрагменты, а потом извлечем собственно адреса, помеченные атрибутом href тега &lt;a&gt;.

In [21]:
# Теперь выделим только то, что взято в тег h3.
beau = BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib")
tags = beau.find_all("a", attrs={'class': 'card-full-other'})
# Формируем список ссылок. Для этого берем первую (кстати, единственную) ссылку из каждого выделенного
# фрагмента, у нее берем значение параметра href. Так как ссылки внутренние, добавляем к ним адрес сайта.
links = ["https://lenta.ru"+l["href"] for l in tags]
print(links)

['https://lenta.ru/articles/2018/08/25/shishkan/', 'https://lenta.ru/photo/2018/08/25/chicken/', 'https://lenta.ru/photo/2018/08/25/ukrparad/', 'https://lenta.ru/articles/2018/08/25/dzyuba/', 'https://lenta.ru/articles/2018/08/25/berlin/', 'https://lenta.ru/photo/2018/08/25/bookstores/', 'https://lenta.ru/photo/2018/08/25/school_children/', 'https://lenta.ru/brief/2018/08/25/slepakovvsshnur/']


In [20]:
h3s

[]

Если теперь написать функцию, которая будет перебирать все адреса и получать из них тексты новостей, то мы получим все новости за определенные сутки. Но это мы сделаем чть позже, а пока просто оформим код загрузки статьи в виде функции.

In [12]:
# Загрузка статьи по URL.
def getOneLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html5lib") 
    # Получаем заголовок статьи.
    aTitle=bs.h1.text.replace("\xa0", " ")
    # Получаем текст статьи.
    anArticle=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    # Возвращаем кортеж из заголовка и текста статьи.
    return aTitle, anArticle


### XPath
[Ссылка 1](https://habr.com/ru/post/526774/)

[Ссылка 2](https://habr.com/ru/post/464897/)

XPath позволяет задать шаблон для пути от корня XML-дерева к интересующей нас вершине.

- . - корень XML-дерева
- / - переход на один уровень ниже.
- // - переход на ноль или больше уровней вниз.
- \* - любая вершина.
- xyz - название вершины.
- [@feature] - вершина с параметром feature.
- [@feature='111'] - вершина с параметром feature, равным "111".
- xyz[n] - n-ый потомок вершины xyz.

А теперь давайте посмотрим как мы можем при помощи XPath обрабатывать HTML-документы.

In [17]:
from lxml import html

In [13]:
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')

In [14]:
tree = html.fromstring(page.text)
print(tree.xpath(".//h1")[0].text_content())
print(tree.xpath(".//a[contains(@class, 'topic-header__time')]")[0].text_content().strip())
print(tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip(), '\n')
print(tree.xpath(".//meta[@property='og:description']")[0].get("content"), '\n')

for p in tree.xpath(".//div[@itemprop='articleBody']/p"):
    print(p.text_content())

NameError: name 'html' is not defined

А теперь та же страница, но через BeautyfulSoup.

In [15]:
souped = BeautifulSoup(page.text)

title = souped("h1")[0].get_text()
when = souped.find_all("a", attrs={'class': 'topic-header__time'})[0].get_text().strip()
author = souped.find_all("div", attrs={'class': 'topic-authors'})[0].get_text()
description = souped.find_all("meta", attrs={'property': 'og:description'})[0]["content"]

for p in souped.find_all("div", attrs={'class': 'topic-body'})[0]("p", attrs={'class':"topic-body__content-text"}): 
    print(p.get_text())
    


Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.
Ранее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и поэтому яблоко желательно съедать вместе с огрызком.


In [16]:
title, when, author

('Обнаружен неожиданный эффект от употребления яблок',
 '15:35, 27 февраля 2021',
 'Соня Кошечкина')

### Упражнение - достать страницы с другого сайта

In [18]:
page = requests.get('https://www.rbc.ru/economics/03/03/2021/603f848c9a7947bdd444102f?from=from_main_1')

### Cookies

Библиотека также позволяет работать с куки. 

In [22]:
le = requests.get("https://lenta.ru")
print(le.cookies)

<RequestsCookieJar[<Cookie is_mobile=0 for .lenta.ru/>, <Cookie lid=vAsAAK5IHGWkB66YAUwNCwB= for .lenta.ru/>, <Cookie lids=482007A4122B60EF for .lenta.ru/>]>


In [23]:
c1 = le.cookies
# c1
c1.set('sdjkfsa', '123445') 
# c1

Cookie(version=0, name='sdjkfsa', value='123445', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [25]:
coo = requests.cookies.RequestsCookieJar()
coo.set("asfdasfd", "12341234")
res = requests.get("https://lenta.ru", cookies=coo)

### Заголовки
При работе с http, мы обмениваемся с сервером заголовками. Например, сервер рассказывает свою версию, какого вида данные он нам вернул, какие он поддерживает протоколы.

In [26]:
res.headers

{'Server': 'nginx', 'Date': 'Tue, 03 Oct 2023 17:07:49 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=50', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Download-Options': 'noopen', 'X-Permitted-Cross-Domain-Policies': 'none', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'P3P': 'CP="This Is Potato!", CP="NON DSP NID ADMa DEVa TAIa PSAa PSDa OUR IND UNI COM NAV"', 'ETag': 'W/"e160d989fd0458a680d75f9ef61049dc"', 'Cache-Control': 'max-age=0, private, must-revalidate', 'X-Request-Id': 'a650c351-6bb8-44fc-8cdd-1052f05ef1c5', 'X-Runtime': '1.142159', 'Set-Cookie': 'is_mobile=0; path=/; domain=.lenta.ru, lids=48210735129986ED;path=/;Max-Age=1800;domain=.lenta.ru, lid=vAsAAGVKHGU1B4lzAeLtCgB=; expires=Thu, 31-Dec-37 23:55:55 GMT; domain=.lenta.ru; path=/', 'Access-Control-Allow-Origin': '*', 'Content-Encoding': 'br'}

Мы тоже можем ему передать заголовки с интересной для нас информацией (можно посмотреть в отладочной информации браузера). Например, наша программа может начать представляться другим браузером.

In [47]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 YaBrowser/20.9.3.136 Yowser/2.5 Safari/537.36'}

response = requests.get('https://lenta.ru', headers=headers)

### Сессия

Если вам необходимо ввести пароль, а потом работать как зарегистрированный пользователь, лучше использовать сессию, которая запомнит все данные. Об этом можно почитать [здесь](https://requests.readthedocs.io/en/latest/user/advanced/).

### Работа с бинарными файлами

Попробуем загрузить pdf-файл со статьей с сайта Киберленинка.

Правда, если мы будем загружать его все вместе с одного IP-адреса, то нас могут забанить за массовое скачивание.

In [27]:
pdf1 = requests.get("https://cyberleninka.ru/article/n/semantiko-sintaksicheskoe-opisanie-glagolov-professionalnoy-deyatelnosti-iz-materialov-k-semantiko-grammaticheskomu-slovaryu/pdf")

Можно заметить, что у объекта есть несколько полей, хранящих бинарные строки, например, `content` и `raw`. Это, собственно, содержимое загруженного файла.

Еще у него есть свойства на случай, если был загружен json (`json`) или текст (мы его уже использовали - `text`).

In [28]:
dir(pdf1)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

Взглянем на содержимое загруженного файла. В первых байтах виден заголовок pdf-файла с автором, использованной программой и другими данными.

In [29]:
pdf1.content[:200]

b"%PDF-1.4\n%\xe2\xe3\xcf\xd3\n1 0 obj\n<</Author(katya)/CreationDate(D:20200129093126+03'00')/Creator(Adobe InDesign CC 14.0 \\(Macintosh\\))/ModDate(D:20200224020154+03'00')/Producer(Acrobat Distiller 15.0 \\(Macintosh"

Поставим себе библиотеку для чтения pdf-файлов и посмотрим на содержимое первой страницы.

In [27]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable


In [30]:
from pypdf import PdfReader
import io

Библиотека принимает на вход открытый файл, но при помощи `io.BytesIO()` мы сможем имитировать чтение из файла, которое будет брать данные из памяти.

In [24]:
f = io.BytesIO(pdf1.content)

reader = PdfReader(f)

Итак, посмотрим на текстовое содержимое первой страницы.

In [25]:
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [26]:
text

'УЧЕНЫЕ  ЗАПИСКИ  ПЕТРОЗАВОДСКОГО  ГОСУДАРСТВЕННОГО  УНИВЕРСИТЕТА\n© Смирнова  Е. А., 2020Т. 42. № 1. С. 72–81 Языкознание  2020\nDOI: 10.15393/uchz.art.2020.435УДК  81`1\nЕКАТЕРИНА  АНДРЕЕВНА  СМИРНОВА\nкандидат  филологических  наук , научный  сотрудник  секто -\nра теоретической  семантики\nФедеральное  государственное  бюджетное  учреждение  нау-\nки Институт  русского  языка  имени  В. В. Виноградова  РАН  \n(Москва , Российская  Федерация )\nkatarzina@yandex.ru\nСЕМАНТИКО -СИНТАКСИЧЕСКОЕ  ОПИСАНИЕ  ГЛАГОЛОВ  \nПРОФЕССИОНАЛЬНОЙ  ДЕЯТЕЛЬНОСТИ  \n(из материалов  к Семантико -грамматическому  словарю )*\nПредставлено  описание  парадигматических  возможностей  глагольной  группы  профессиональ -\nной деятельности  на -ничать  и выявлены  семантико -синтаксические  запреты  на образование  тех \nили иных  глагольных  форм . На материале  Национального  корпуса  русского  языка  (НКРЯ ) показа -\nны тенденции  в области  глагольного  словоизменения  и словообразования  внутри  данной  

М-да... С разбивкой на строки и абзацы полная беда. Попробуем хотя бы устранить преносы.

In [27]:
text.replace(' -\n', '').replace('-\n', '')

'УЧЕНЫЕ  ЗАПИСКИ  ПЕТРОЗАВОДСКОГО  ГОСУДАРСТВЕННОГО  УНИВЕРСИТЕТА\n© Смирнова  Е. А., 2020Т. 42. № 1. С. 72–81 Языкознание  2020\nDOI: 10.15393/uchz.art.2020.435УДК  81`1\nЕКАТЕРИНА  АНДРЕЕВНА  СМИРНОВА\nкандидат  филологических  наук , научный  сотрудник  сектора теоретической  семантики\nФедеральное  государственное  бюджетное  учреждение  науки Институт  русского  языка  имени  В. В. Виноградова  РАН  \n(Москва , Российская  Федерация )\nkatarzina@yandex.ru\nСЕМАНТИКО -СИНТАКСИЧЕСКОЕ  ОПИСАНИЕ  ГЛАГОЛОВ  \nПРОФЕССИОНАЛЬНОЙ  ДЕЯТЕЛЬНОСТИ  \n(из материалов  к Семантико -грамматическому  словарю )*\nПредставлено  описание  парадигматических  возможностей  глагольной  группы  профессиональной деятельности  на -ничать  и выявлены  семантико -синтаксические  запреты  на образование  тех \nили иных  глагольных  форм . На материале  Национального  корпуса  русского  языка  (НКРЯ ) показаны тенденции  в области  глагольного  словоизменения  и словообразования  внутри  данной  группы : \nзапо

### Потоковое аудио
Теперь попробуем работать с потоковым аудио. Я начал со страницы http://radio.garden , которая хранит ссылки на интернет-радиостанции. С ее помощью я вышел вот на этот адрес. Попробуем грузить с него потоковое аудио в файл.

In [31]:
stream_url = 'https://radiorecord.hostingradio.ru/198096.aacp?listening-from-radio-garden=1680922353'
r = requests.get(stream_url, stream=True)

with open('stream.mp3', 'wb') as f:
    try:
        for block in r.iter_content(1024):
            f.write(block)
    except KeyboardInterrupt:
        pass

А попробуем теперь прослушать это аудио.

In [53]:
!pip install pydub

Defaulting to user installation because normal site-packages is not writeable


In [32]:
from pydub import AudioSegment
from pydub.playback import play

In [33]:
stream_url = 'http://stream1.waszeradiofm.pl:8000/;?listening-from-radio-garden=1696354810'
r = requests.get(stream_url, stream=True)

try:
    for block in r.iter_content(50240):
        f = io.BytesIO(block)
        sound = AudioSegment.from_file(f)
        play(sound)
except KeyboardInterrupt:
    pass

Input #0, wav, from '/tmp/tmp8ikkd76p.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:03.16, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmpa8ctlp_b.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:03.13, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmp7urvghlr.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:03.13, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmpp24_0u67.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:03.13, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s
